In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
#datos del censo para la densidad de población:

poblacion = pd.read_excel('../Data/Raw/poblacion_2023.xlsx')

In [3]:
poblacion.head(1)

,Provincia,Municipio,Unidad Poblacional,Total 2023,Hombres 2023,Mujeres 2023
0,1,1,000000 ALEGRÍA-DULANTZI,2975,1539,1436


In [4]:
codigo = []
municipio = []
for x in poblacion['Unidad Poblacional']:
    codigo.append(x.split(' ',1)[0])
    municipio.append(x.split(' ',1)[1])
    
poblacion['código_unidad'] = codigo
poblacion['mun_nom'] = municipio
poblacion = poblacion[poblacion['código_unidad'] == '000000']
poblacion.drop(columns=['Unidad Poblacional'], inplace = True)

In [5]:
poblacion.reset_index(inplace = True, drop = True)

In [6]:
for x in range(len(poblacion['Municipio'])):
    poblacion['Municipio'][x] = str(poblacion['Municipio'][x]).zfill(3)

/var/folders/ps/6ygxcm2s5xj2_hmh0rwz7d640000gn/T/ipykernel_68708/2587515013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poblacion['Municipio'][x] = str(poblacion['Municipio'][x]).zfill(3)


In [7]:
for x in range(len(poblacion['Provincia'])):
    poblacion['Provincia'][x] = str(poblacion['Provincia'][x])

/var/folders/ps/6ygxcm2s5xj2_hmh0rwz7d640000gn/T/ipykernel_68708/2221374254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poblacion['Provincia'][x] = str(poblacion['Provincia'][x])


In [8]:
poblacion.head(1)

,Provincia,Municipio,Total 2023,Hombres 2023,Mujeres 2023,código_unidad,mun_nom
0,1,001,2975,1539,1436,000000,ALEGRÍA-DULANTZI


In [9]:
poblacion['Cod. INE']=poblacion['Provincia']+poblacion['Municipio']+poblacion['código_unidad']
poblacion.columns = poblacion.columns.str.replace(' ','_').str.lower().str.replace('.','')
poblacion.drop(columns=['código_unidad'], inplace = True)
poblacion.columns

Index(['provincia', 'municipio', 'total_2023', 'hombres_2023', 'mujeres_2023',
       'mun_nom', 'cod_ine'],
      dtype='object')

In [10]:
test = poblacion.copy()

In [11]:
replace_dict = {r'(\b\w*) \(LA\)':'LA ',
                r'(\b\w*) \(EL\)':'EL ',
               r'(\b\w*) \(LAS\)':'LAS ',
               r'(\b\w*) \(LOS\)':'LOS ',
               r"(\b\w*) \(L'\)":"L'",
               r'(\b\w*) \(A\)':'A ',
               r'(\b\w*) \(O\)':'O ',
               r'(\b\w*) \(OS\)':'OS ',
                r'(\b\w*) \(AS\)':'AS ',
                'r(\b\w*) \(ELS\)': 'ELS ',
                'r(\b\w*) \(ES\)':'ES ', 
                r'(\b\w*) \(LES\)':'LES ', 
                r'(\b\w*) \(SA\)':'SA ',
                r'(\b\w*) \(SES\)':'SES '
               }

In [12]:
muns=[]
for x in poblacion['mun_nom']:
    if re.search(r'\b\w* \(LA\)', x, flags=re.IGNORECASE):
            mun = 'LA ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(LAS\)', x, flags=re.IGNORECASE):
            mun = 'LAS ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(LOS\)', x, flags=re.IGNORECASE):
            mun = 'LOS ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(EL\)', x, flags=re.IGNORECASE):
            mun = 'EL ' + x
            muns.append(mun)
    elif re.search(r"\b\w* \(L'\)", x, flags=re.IGNORECASE):
            mun = "L'" + x
            muns.append(mun)
    elif re.search(r'\b\w* \(A\)', x, flags=re.IGNORECASE):
            mun = 'A ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(O\)', x, flags=re.IGNORECASE):
            mun = 'O ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(AS\)', x, flags=re.IGNORECASE):
            mun = 'AS ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(OS\)', x, flags=re.IGNORECASE):
            mun = 'OS ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(ELS\)', x, flags=re.IGNORECASE):
            mun = 'ELS ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(ES\)', x, flags=re.IGNORECASE):
            mun = 'ES ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(SA\)', x, flags=re.IGNORECASE):
            mun = 'SA ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(LES\)', x, flags=re.IGNORECASE):
            mun = 'LES ' + x
            muns.append(mun)
    elif re.search(r'\b\w* \(SES\)', x, flags=re.IGNORECASE):
            mun = 'SES ' + x
            muns.append(mun)
    else:
        muns.append(x)

In [13]:
poblacion['municipios']=muns

In [14]:
poblacion.drop(columns=['mun_nom'], inplace = True)

In [15]:
poblacion['municipios'] = poblacion['municipios'].str.capitalize()
replace = {'á':'a', 'à':'a',
           'é':'e', 'è':'e',
           'í':'i', 'ì':'i', 
           'ó':'o', 'ò':'o', 'ô':'o', 
           'ú':'u', 
           'Á':'A', 'À':'A',
           'Ó':'O', 
           'Ú':'U', 
           'É':'E', 
           'Ì':'I',
           'Í':'I',
           #r'^(.*\/)':'',
           r'\([^\)]*\)':''
          }
poblacion.replace(replace, regex=True, inplace = True)
poblacion['municipios'] = poblacion['municipios'].str.strip()
poblacion['municipios'] = poblacion['municipios'].str.capitalize()
poblacion.head(3)

,provincia,municipio,total_2023,hombres_2023,mujeres_2023,cod_ine,municipios
0,1,001,2975,1539,1436,1001000000,Alegria-dulantzi
1,1,002,10313,5134,5179,1002000000,Amurrio
2,1,003,1409,732,677,1003000000,Aramaio


In [16]:
table_2 = pd.read_csv('../Data/Raw/superficies.csv')

In [17]:
sup_2 = pd.DataFrame(table_2)

In [18]:
sup_2.shape

(8132, 9)

In [19]:
sup_2.drop(columns=['Municipio#', 'Comarca', 'Altitud<br/>(m.s.n.m.)', 'PoblaciÃ³n<br/>(2019)', 'Densidad<br/>(hab./kmÂ²)'], inplace = True)

In [23]:
sup_2.rename(columns={'municipio':'municipios','Superficie<br/>(kmÂ²)':'km2', 'CC.AA.':'comunidad'}, inplace = True)
sup_2.columns = sup_2.columns.str.lower().str.strip()
sup_2.columns

Index(['municipios', 'provincia', 'comunidad', 'km2'], dtype='object')

In [24]:
replacement = {'Ã¡':'a','Ã©': 'e','Ã³':'o','Ã²':'o', 'Ã¨':'e','Ã':'e', 'Ã±':'ñ','Ãº':'u','Ã':'i'}
sup_2.replace(replacement, inplace = True, regex = True)

In [25]:
sup_2.municipios = sup_2.municipios.str.strip()
sup_2.municipios = sup_2.municipios.str.capitalize()
sup_2.tail(2)

,municipios,provincia,comunidad,km2
8130,Llocnou de la corona,Provincia de Valencia,Comunidad Valenciana,004
8131,Emperador (valencia),Provincia de Valencia,Comunidad Valenciana,003


In [28]:
sup_2['km2'] = sup_2['km2'].str.replace('.', '')
sup_2['km2'] = sup_2['km2'].str.replace(',', '.')

In [29]:
sup_2['km2'] = sup_2['km2'].astype(float)

In [30]:
type(sup_2['km2'][8130])

numpy.float64

In [31]:
#eliminamos más caracteres regex que luego nos dan duplicados al hacer merge (reducimos de casi mil a menos de 450)
more_replacements = {r'i\xad':'i',
                    r'I\x89':'E',
                    r'I\x93':'O',
                    r'i\x81':'a',
                    r'I\x81':'A',
                    r'I\x80':'A',
                    r'i\x80':'a',
                    r'i½':'y',
                    r'i\xa0':'a',
                    r'\([^\)]*\)':'',
                    r'i¼':'ü',
                    r'i§':'s',
                    r'i\x93':'o',
                    r'i\x9a':'u',
                    r'i\x8d':'i',
                     r'I\x8d':'I',
                    r'I\x92':'O',
                    r'I\x9a':'U',
                    'ii':'ia',
                    r'i¯':'ï',
                    }

In [32]:
sup_2.replace(more_replacements, regex = True, inplace = True)
sup_2.municipios = sup_2.municipios.str.strip()

In [33]:
pop_den = pd.merge(poblacion,sup_2, how='outer', on=['municipios'])
pop_den

,provincia_x,municipio,total_2023,hombres_2023,mujeres_2023,cod_ine,municipios,provincia_y,comunidad,km2
0,1,001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,NaN,NaN,NaN
1,1,002,10313.0,5134.0,5179.0,1002000000,Amurrio,Provincia de Araba,Pais Vasco,9635.0
2,1,003,1409.0,732.0,677.0,1003000000,Aramaio,Provincia de Araba,Pais Vasco,7327.0
3,1,004,1832.0,898.0,934.0,1004000000,Artziniega,Provincia de Araba,Pais Vasco,2729.0
4,1,006,232.0,120.0,112.0,1006000000,Armiñon,Provincia de Araba,Pais Vasco,1294.0
...,...,...,...,...,...,...,...,...,...,...
8463,NaN,NaN,NaN,NaN,NaN,NaN,Barañain,Provincia de Navarra,Comunidad Foral de Navarra,139.0
8464,NaN,NaN,NaN,NaN,NaN,NaN,Villava,Provincia de Navarra,Comunidad Foral de Navarra,106.0
8465,NaN,NaN,NaN,NaN,NaN,NaN,Sant hipolit de voltregi,Provincia de Barcelona,Cataluña,9.0
8466,NaN,NaN,NaN,NaN,NaN,NaN,Ayacor,Provincia de Valencia,Comunidad Valenciana,89.0


In [34]:
pop_den.isnull().sum()

provincia_x     305
municipio       305
total_2023      305
hombres_2023    305
mujeres_2023    305
cod_ine         305
municipios        0
provincia_y     304
comunidad       304
km2             304
dtype: int64

In [35]:
pop_den.shape

(8468, 10)

In [36]:
#sigue habiendo duplicados, así que vamos sacando listas de reemplazo para modificar los dataframes antes de hacer merge

In [37]:
no_match = pop_den[pop_den['cod_ine'].isna()]
no_match.municipios.unique()

array(['Exeya', 'Samianigo', 'Medina-sidonia', 'Elche',
       'Alcoba de los montes', 'Sarinyena', 'Valle de yerri',
       'Palma de mallorca', 'Alicante', 'Alcali de xivert', 'Jijona',
       'Santa eulalia del rio', 'Monovar', 'Pollensa',
       'Vistabella del maestrazgo', 'Mogente', 'Isaba', 'Arce',
       'Lumbreras', 'Isona i conca delli', 'Arti', 'Lucena del cid',
       'Les coves de vinromi', 'La pobla de benifassi', 'Cotobade',
       'Sagunto', 'Alcoy', 'Ochagavia', 'Pinoso', 'Ribera alta', 'Ujue',
       'Castellon', 'La laguna', 'Figols i alinyi', 'Mellid', 'Navascues',
       'Leoz', 'Albanyi', 'Villafranca del cid', 'Tori', 'Romanzado',
       'Longuida', 'Peralta', 'Olite', 'Llangreu', 'Chert',
       'Sant llorens des cardassar', 'Orbaitzeta', 'Arcas del villar',
       'Santa eulalia del campo', 'Useras', 'Cerdedo', 'Benasal',
       'Peñiscola', 'Guesalaz', 'Zahara de la sierra', 'Lobera',
       'Hondon de las nieves', 'Javea', 'Oristi', 'Masanet de cabrenys',
   

In [38]:
no_match_2 = pop_den[pop_den['km2'].isna()]
no_match_2.municipios.unique()

array(['Alegria-dulantzi', 'Arratzua-ubarrundia',
       'Moreda de alava/moreda araba', 'Erriberagoitia/ribera alta',
       'Erriberabeitia', 'Agurain/salvatierra', "L'atzubia", 'Alcosser',
       'Alcoi/alcoy', 'Alacant/alicante',
       'El poble nou de benitatxell /benitachell', 'Calp',
       'El camp de mirra /campo de mirra', 'Elx/elche', 'Fageca',
       'El fondo de les neus /hondon de las nieves', 'Xalo',
       'Xabia/javea', 'Xixona/jijona', "L'orxa /lorcha",
       'Monover/monovar', 'El pinos /pinoso',
       'Sant vicent del raspeig/san vicente del raspeig',
       'La torre de les maçanes /torremanzanas', "La vall d'alcala",
       'La vall de gallinera', 'La vila joiosa /villajoyosa',
       'Manjabalago y ortigosa de rioalmar', 'Arta', 'Deia', 'Eivissa',
       'Palma', 'Pollença', 'Sant llorenç des cardassar',
       'Santa eularia des riu', 'Vilafranca de bonany',
       "L'ametlla del valles", 'Argençola', 'Baga', 'Balenya',
       'Bigues i riells del fai', 'Borr

In [39]:
cambios = {}
cambios = pd.DataFrame(cambios)
cambios['original'] = no_match_2.municipios.unique()
cambios[['c1','c2']] = cambios['original'].str.split('/', expand = True)
cambios.c2 = cambios.c2.str.strip().str.capitalize()
cambios.head(2)

,original,c1,c2
0,Alegria-dulantzi,Alegria-dulantzi,None
1,Arratzua-ubarrundia,Arratzua-ubarrundia,None


In [40]:
cambios_b = {}
cambios_b = pd.DataFrame(cambios_b)
cambios_b['c1'] = pd.DataFrame(no_match.municipios.unique().tolist())
cambios_b['c2'] = pd.DataFrame(no_match.municipios.unique().tolist())
cambios_b['c3'] = pd.DataFrame(no_match.municipios.unique().tolist())
cambios_b.head(2)


,c1,c2,c3
0,Exeya,Exeya,Exeya
1,Samianigo,Samianigo,Samianigo


In [41]:
#merge primero con columna c2:
cambios_poblacion_1 = pd.merge(cambios, cambios_b, on='c2')
cambios_poblacion_1.head(5)

,original,c1_x,c2,c1_y,c3
0,Erriberagoitia/ribera alta,Erriberagoitia,Ribera alta,Ribera alta,Ribera alta
1,Alcoi/alcoy,Alcoi,Alcoy,Alcoy,Alcoy
2,Alacant/alicante,Alacant,Alicante,Alicante,Alicante
3,El poble nou de benitatxell /benitachell,El poble nou de benitatxell,Benitachell,Benitachell,Benitachell
4,El camp de mirra /campo de mirra,El camp de mirra,Campo de mirra,Campo de mirra,Campo de mirra


In [42]:
cambios_poblacion_1.drop(columns=['c1_x','c2', 'c1_y'], inplace = True)

In [43]:
original = cambios_poblacion_1['original'].tolist()
c3 = cambios_poblacion_1['c3'].tolist()

In [44]:
#merge ahora con columna c1:
cambios_poblacion_2 = pd.merge(cambios, cambios_b, on='c1')
cambios_poblacion_2.drop(columns=['c2_x','c1', 'c2_y'], inplace = True)
cambios_poblacion_2.head(2)

,original,c3
0,Moreda de alava/moreda araba,Moreda de alava
1,Sant vicent del raspeig/san vicente del raspeig,Sant vicent del raspeig


In [45]:
original.extend(cambios_poblacion_2['original'].tolist())
c3.extend(cambios_poblacion_2['c3'].tolist())

In [46]:
cambios_dict = dict(zip(original,c3))

In [47]:
poblacion.replace(cambios_dict, inplace = True)
poblacion['municipios'] = poblacion['municipios'].str.strip()
poblacion['municipios'] = poblacion['municipios'].str.capitalize()

In [48]:
#repetimos merge para ver con cuántos valores únicos nos quedamos ahora:
pop_den = pd.merge(poblacion,sup_2, how='outer', on=['municipios'])
pop_den.tail(5)

,provincia_x,municipio,total_2023,hombres_2023,mujeres_2023,cod_ine,municipios,provincia_y,comunidad,km2
8365,NaN,NaN,NaN,NaN,NaN,NaN,Benisuera,Provincia de Valencia,Comunidad Valenciana,212.0
8366,NaN,NaN,NaN,NaN,NaN,NaN,Cerdi,Provincia de Valencia,Comunidad Valenciana,152.0
8367,NaN,NaN,NaN,NaN,NaN,NaN,Sant hipolit de voltregi,Provincia de Barcelona,Cataluña,9.0
8368,NaN,NaN,NaN,NaN,NaN,NaN,Ayacor,Provincia de Valencia,Comunidad Valenciana,89.0
8369,NaN,NaN,NaN,NaN,NaN,NaN,Benirredri,Provincia de Valencia,Comunidad Valenciana,39.0


In [49]:
no_match_2 = pop_den[pop_den['km2'].isna()]
no_match_2 = no_match_2.drop_duplicates(subset=['municipios'], keep=False)
no_match_2.municipios.unique()

array(['Alegria-dulantzi', 'Arratzua-ubarrundia', 'Erriberabeitia',
       'Agurain/salvatierra', "L'atzubia", 'Alcosser', 'Calp', 'Fageca',
       'Xalo', "La vall d'alcala", 'La vall de gallinera',
       'Manjabalago y ortigosa de rioalmar', 'Arta', 'Deia', 'Eivissa',
       'Palma', 'Pollença', 'Sant llorenç des cardassar',
       'Santa eularia des riu', 'Vilafranca de bonany',
       "L'ametlla del valles", 'Argençola', 'Baga', 'Balenya',
       'Bigues i riells del fai', 'Borreda', 'Canyelles', 'Castellterçol',
       'Cornella de llobregat', "L'espunyola", "L'estany", 'Gava',
       'Guardiola de bergueda', "L'hospitalet de llobregat",
       "Lliça d'amunt", 'Lliça de vall', 'Lluça',
       'Les masies de voltrega', 'La nou de bergueda', 'Òdena', 'Orista',
       'Òrrius', 'Palau-solita i plegamans', 'Palleja', 'Polinya',
       'Prats de lluçanes', 'Sant agusti de lluçanes',
       'Sant boi de lluçanes', 'Sant hipolit de voltrega',
       'Sant jaume de frontanya', "Sant llo

In [50]:
no_match = pop_den[pop_den['cod_ine'].isna()]
no_match = no_match.drop_duplicates(subset=['municipios'], keep=False)
no_match.municipios.unique()

array(['Exeya', 'Samianigo', 'Medina-sidonia', 'Alcoba de los montes',
       'Sarinyena', 'Palma de mallorca', 'Alcali de xivert',
       'Santa eulalia del rio', 'Pollensa', 'Vistabella del maestrazgo',
       'Lumbreras', 'Isona i conca delli', 'Arti', 'Les coves de vinromi',
       'La pobla de benifassi', 'Cotobade', 'Castellon', 'La laguna',
       'Figols i alinyi', 'Mellid', 'Albanyi', 'Tori', 'Llangreu',
       'Chert', 'Sant llorens des cardassar', 'Orbaitzeta',
       'Arcas del villar', 'Santa eulalia del campo', 'Cerdedo',
       'Benasal', 'Zahara de la sierra', 'Lobera', 'Oristi',
       'Masanet de cabrenys', 'La ametlla de mar', 'San mateo',
       'Veracruz', 'Santa maria de corco', 'Guardiola de berguedi',
       'Oyarzun', 'Las rozas', 'Arrazua-ubarrundia',
       'Espluga de francoli', 'San lorenzo del escorial', 'Vilaflor',
       'Montelli i martinet', 'Samartin del rei aurelio',
       'Vall de gallinera', 'Llusi', 'Egües', 'Sant carles de la rapita',
       'Od

In [63]:
more_replacements = {'La a':"L'a",
                    "L' ":"L'",
                     'Eivissa':'Ibiza',
                     'ç':'s',
                     'Salvatierra/agurain':'Agurain/salvatierra',
                     'Polinyi':'Polinya',
                     'de el':'del',
                     'Estany':"L'estany",
                     "Hospitalet de llobregat":"L'hospitalet de llobregat",
                     'Cerdi':'Cerda',
                     'Llorens del penedes':'Llorens del penedes',
                     'La escala':"L'escala",
                     'Samianigo':'Sabiñanigo',
                     'Exeya':'Ejea',
                     'Estella/lizarra':'Estella-lizarra',
                     'Arrazua':'Arratzua',
                     'Erriberabeitia':'Ribera baja',
                     'soliti':'solita',
                     'Medina-sidonia':'Medina sidonia',
                     'Alcoba de los montes':'Alcoba',
                     'Sarinyena':'Sariñena',
                     'Alcali':'Alcala',
                     'Santa eularia des riu':'Santa eulalia del rio',
                     'Pollensa':'Pollensa',
                     'Vistabella del maestrat':'Vistabella del maestrazgo',
                     'Lumbreras de cameros':'Lumbreras',
                     'delli':'della',
                     'vinromi':'vinroma',
                     'benifassi':'benifassa',
                     'Castello de la plana':'Castellon',
                     'San cristobal de la laguna':'La laguna',
                     'alinyi':'alinya',
                     'Mellid':'Melide',
                     'Albanyi':'Albanya',
                     'Llangreu':'Langreo',
                     'Xert':'Chert',
                     'Sant llorens des cardassar':'Sant llorens des cardassar',
                     'Orbaitzeta':'Orbaizeta',
                     'Arcas del villar':'Arcas',
                     'Benassal':'Benasal',
                     'Oristi':'Orista',
                     'Masanet de cabrenys':'Masanet de cabrenys',
                     "L'ametlla de mar":"L'ametlla de mar",
                     "L'ametlla del valles":"L'ametlla del valles",
                     'Sant mateu':'San mateo',
                     'Santa maria de corco':"L'esquirol",
                     'berguedi':'bergueda',
                     'Ejea de los caballeros':'Ejea',
                     'empordi':'emporda',
                     'Puigcerdi':'Puigcerda',
                     'de jarama':'del jarama',
                     'Las rozas de madrid':'Las rozas',
                     'Alegria de alava':'Alegria-dulantzi',
                     'Espluga calva':"L'espluga calba",
                     'Vilaflor de chasna':'Vilaflor',
                     'Montelli':'Montella',
                     'Samartin del rei aurelio':'San martin del rey aurelio',
                     'Vall de gallinera':'La vall de gallinera',
                     'Llusi':'Llusa',
                     'Egües':'Valle de egües/eguesibar',
                     'La rapita':'Sant carles de la rapita',
                     'Òdena':'Odena',
                     'Deba':'Deva',
                     'Cassi':'Cassa',
                     'Celri':'Celra',
                     'Foixi':'Foixa',
                     'Cornelli':'Cornella',
                     'Pozorrubio de santiago':'Pozorrubio',
                     'Borredi':'Borreda',
                     'Benlloch':'Benlloc',
                     'Porquera':'Porqueira',
                     'Bagi':'Baga',
                     'Jarque de moncayo':'Jarque',
                     'Legazpia':'Legazpi',
                     'Uharte arakil':'Uharte-arakil',
                     'Ollo':'Valle del ollo',
                     'Valle de ollo/ollaran':'Valle del ollo',
                     'Brunyola i sant marti sapresa':'Brunyola',
                     'Espunyola':"L'espunyola",
                     'Vidri':'Vidra',
                     'Ribera baja/erribera beitia':'Ribera baja',
                     'Albi':"L'albi",
                     'Calonge i sant antoni':'Calonge',
                     'Almassora':'Almazora',
                     'Maldi':'Malda',
                     'Etxarri aranatz':'Etxarri-aranatz',
                     'Olleria':"L'olleria",
                     'La cisterniga':'Cisterniga',
                     'Pardinas':'Pardines',
                     'Gavi':'Gava',
                     'Bigues i riells del fai':'Bigues i riells',
                     'Llansi':'Llansa',
                     'Barberi':'Barbera',
                     'Herbes':'Herbers',
                     'Calpe':'Calp',
                     'Calp':'Calpe',
                     'Aleixar':"L'aleixar",
                     'Guimeri':'Guimera',
                     'Albages':"L'albages",
                     'Massanas':'Massanes',
                     'Pradales':'Carabias',
                     'Villa de otura':'Otura',
                     'Vilafranca':'Villafranca',
                     'alcali':'alcala',
                     'Arrankudiaga-zollo':'Arrankudiaga',
                     'Llisi':'Llisa',
                     'voltregi':'voltrega',
                     'matajudios':'mota de judios',
                     'Vistabella de huerva':'Vistabella',
                     'frontanyi':'frontanya',
                     'La serratella':'Serratella',
                     'Sarratella':'Serratella',
                     'La lantejuela':'Lantejuela',
                     'nyi':'nya',
                     'Manjabalago y ortigosa de rioalmar':'Manjabalago',
                     'Roda de bari':'Roda de bara',
                     'Roda de bera':'Roda de bara',
                     'Abanto y ciervana-abanto zierbena':'Abanto y ciervana',
                     'Marsi':'Marsa',
                     'Corsi':'Corsa',
                     'Castellseri':'Castellsera',
                     'Esponelli':'Esponella',
                     'Deya':'Deia',
                     'El genoves':'Genoves',
                     'Adsubia':"L'atzubia",
                     'Savalli':'Savalla',
                     'de alfara':"d'alfara",
                     'Soraluze-placencia de las armas':'Placencia de las armas',
                     'Canyellas':'Canyelles',
                     'Torms':'Els torms',
                     'Bidegoian':'Bidania-goiatz',
                     'Valle de trapaga-trapagaran':'Valle de trapaga',
                     'Tortelli':'Tortella',
                     'Ciurana':'Siurana',
                     'marzi':'marza',
                     'Palleji':'Palleja',
                     'La palma de cervello':'Palma de cervello',
                     'Òrrius':'Orrius',
                     'Espluga':"L'espluga",
                     'Valle de egües/eguesibar':'Valle de egües',
                     'Collado de la vera':'Collado',
                     'Urroz':'Urrotz',
                     'Facheca':'Fageca',
                     'Argentera':"L'argentera",
                     'Borrassi':'Borrassa',
                     'Favareta':'Favara',
                     'Oiz':'Oitz',
                     'Palomar':'El palomar',
                     'Enova':"L'enova",
                     'Almiseri':'Almisera',
                     'Ulli':'Ulla',
                     'El mili':'El mila',
                     "L'alqueria de la comtessa":'Alqueria de la condesa',
                     'Alcocer de planes':'Alcosser',
                     'Zahara de la sierra':'Zahara',
                     'Santa eulalia del campo':'Santa eulalia',
                     'Lobera':'Lobeira',
                     'Oyarzun':'Oiartzun',
                     'Flasi':'Flasa',
                     'Rotgli i corberi':'Rotgla i corbera',
                     'Parlavi':'Parlava',
                     'Guadasequies':'Guadassequies',
                     'Gainza':'Gaintza',
                     'Torre endomenech':"La torre d'en domenec",
                     'Benisano':'Benissano',
                     'Benirredri':'Benirredra',
                     'El real de gandia':'Real de gandia',
                     'Rafelbuñol':'Rafelbunyol',
                     'El rourell':'Rourell',
                     'Benisuera':'Benissuera',
                     'Jalon':'Xalo',
                     'Veracruz':'Beranuy',
                     'Villanueva de castellon':'Castello'
                    }

#cambiar luego a mano San cristobal de la laguna, Castellón de la Plana, Palma de Mallorca, Tori por Tora,
#Ejea de los caballeros

In [64]:
poblacion.replace(more_replacements, inplace = True, regex = True)

In [65]:
sup_2.replace(more_replacements, inplace = True, regex = True)

In [66]:
replacements_cell = {'Arti':'Arta',
                    'Palma':'Palma de mallorca',
                    'Tori':'Tora',
                    }

In [67]:
sup_2.replace(replacements_cell, inplace = True)

In [68]:
poblacion.replace(replacements_cell, inplace = True)

In [69]:
#repetimos merge para ver con cuántos valores únicos nos quedamos ahora:
pop_den = pd.merge(poblacion,sup_2, how='outer', on=['municipios'])
pop_den.tail(5)

,provincia_x,municipio,total_2023,hombres_2023,mujeres_2023,cod_ine,municipios,provincia_y,comunidad,km2
8162,52,001,85491.0,43047.0,42444.0,52001000000,Melilla,NaN,NaN,NaN
8163,NaN,NaN,NaN,NaN,NaN,NaN,Cotobade,Provincia de Pontevedra,Galicia,1352.0
8164,NaN,NaN,NaN,NaN,NaN,NaN,Cerdedo,Provincia de Pontevedra,Galicia,80.0
8165,NaN,NaN,NaN,NaN,NaN,NaN,Mieras,Provincia de Girona,Cataluña,2614.0
8166,NaN,NaN,NaN,NaN,NaN,NaN,Ayacor,Provincia de Valencia,Comunidad Valenciana,89.0


In [70]:
pop_den.shape

(8167, 10)

In [71]:
#Eliminamos los municipios restantes que no aparecen en la lista del INE
pop_den.dropna(subset=['cod_ine'], inplace = True)

In [72]:
#completamos datos de superficie para Frontera, Ceuta y Melilla
pop_den[(pop_den['municipios'] == 'Frontera') | (pop_den['municipios'] == 'Ceuta') | (pop_den['municipios'] == 'Melilla')]

,provincia_x,municipio,total_2023,hombres_2023,mujeres_2023,cod_ine,municipios,provincia_y,comunidad,km2
5754,38,013,4465.0,2236.0,2229.0,38013000000,Frontera,NaN,NaN,NaN
8161,51,001,83039.0,41894.0,41145.0,51001000000,Ceuta,NaN,NaN,NaN
8162,52,001,85491.0,43047.0,42444.0,52001000000,Melilla,NaN,NaN,NaN


In [73]:
pop_den.iloc[5754,7] = 'Caceres'
pop_den.iloc[5754,8] = 'Castilla-La Mancha'
pop_den.iloc[5754,9] = 34.57
pop_den.iloc[8161,7] = 'Ceuta'
pop_den.iloc[8161,8] = 'Ceuta'
pop_den.iloc[8161,9] = 18.5
pop_den.iloc[8162,7] = 'Melilla'
pop_den.iloc[8162,8] = 'Melilla'
pop_den.iloc[8162,9] = 12.3

In [74]:
pop_den.isna().sum()

provincia_x     0
municipio       0
total_2023      0
hombres_2023    0
mujeres_2023    0
cod_ine         0
municipios      0
provincia_y     0
comunidad       0
km2             0
dtype: int64

In [75]:
pop_den.rename(columns={'provincia_x':'prov_id', 'municipio':'mun_id','cod_ine':'ine_id', 'municipios':'municipio','provincia_y':'provincia'}, inplace = True)
pop_den.head(1)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2
0,1,001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Araba,Pais Vasco,1995.0


In [76]:
pop_den.prov_id =pop_den.prov_id.apply(lambda x: str(x).zfill(2))
pop_den.head(1)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2
0,01,001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Araba,Pais Vasco,1995.0


In [77]:
pop_den['densidad'] = pop_den['total_2023']/pop_den['km2']
pop_den.head(5)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2,densidad
0,01,001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Araba,Pais Vasco,1995.0,1.491228
1,01,002,10313.0,5134.0,5179.0,1002000000,Amurrio,Provincia de Araba,Pais Vasco,9635.0,1.070368
2,01,003,1409.0,732.0,677.0,1003000000,Aramaio,Provincia de Araba,Pais Vasco,7327.0,0.192302
3,01,004,1832.0,898.0,934.0,1004000000,Artziniega,Provincia de Araba,Pais Vasco,2729.0,0.671308
4,01,006,232.0,120.0,112.0,1006000000,Armiñon,Provincia de Araba,Pais Vasco,1294.0,0.179289


In [78]:
pop_den.mun_id = pop_den.prov_id + pop_den.mun_id

In [79]:
pop_den

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2,densidad
0,01,01001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Araba,Pais Vasco,1995.0,1.491228
1,01,01002,10313.0,5134.0,5179.0,1002000000,Amurrio,Provincia de Araba,Pais Vasco,9635.0,1.070368
2,01,01003,1409.0,732.0,677.0,1003000000,Aramaio,Provincia de Araba,Pais Vasco,7327.0,0.192302
3,01,01004,1832.0,898.0,934.0,1004000000,Artziniega,Provincia de Araba,Pais Vasco,2729.0,0.671308
4,01,01006,232.0,120.0,112.0,1006000000,Armiñon,Provincia de Araba,Pais Vasco,1294.0,0.179289
...,...,...,...,...,...,...,...,...,...,...,...
8158,50,50901,167.0,103.0,64.0,50901000000,Biel,Provincia de Zaragoza,Aragon,13073.0,0.012774
8159,50,50902,87.0,45.0,42.0,50902000000,Marracos,Provincia de Zaragoza,Aragon,1692.0,0.051418
8160,50,50903,2866.0,1455.0,1411.0,50903000000,Villamayor de gallego,Provincia de Zaragoza,Aragon,8936.0,0.320725
8161,51,51001,83039.0,41894.0,41145.0,51001000000,Ceuta,Ceuta,Ceuta,18.5,4488.594595


In [80]:
#pop_den.to_csv('densidad_poblacion.csv', encoding='utf-8-sig', index = False)